### Losset DYN - calculate DR indicator (dynamical part only)

In [1]:
import os
import numpy as np
from netCDF4 import Dataset
import xarray as xr
import sys
%xmode Plain

Exception reporting mode: Plain


In [2]:
data_dir = '/gws/nopw/j04/kscale/USERS/dship/ERA5/'

ds_u    = xr.open_dataset(os.path.join(data_dir, 'era5_u_component_of_wind_200501_12hourly.nc'))
ds_v = xr.open_dataset(os.path.join(data_dir, 'era5_v_component_of_wind_200501_12hourly.nc'))
ds_omega = xr.open_dataset(os.path.join(data_dir, 'era5_vertical_velocity_200501_12hourly.nc'))

try:
    ds_u = ds_u.rename({'lon':'longitude','lat':'latitude'})
    ds_v = ds_v.rename({'lon':'longitude','lat':'latitude'})
    ds_omega = ds_omega.rename({'lon':'longitude','lat':'latitude'})
except:
    pass

lon     = ds_u.variables['longitude'][:]
lat     = ds_u.variables['latitude'][:]
lev     = ds_u.variables['level'][:]
time    = ds_u.variables['time'][:]
time    = time[0:2] 
print('lon lat lev time loaded')

lon lat lev time loaded


In [3]:
# Read u, v, and omega data
u = ds_u['u']
print('u loaded')
# Just first 2 timesteps to make script run fast for coparison with Matlab version
u=u[0:2,:,:,:]
# u[1,4,:,:].plot(figsize=(20,10))

u loaded


In [4]:
v = ds_v['v']
print('v loaded')
v=v[0:2,:,:,:]

v loaded


In [5]:
omega = ds_omega['w']  # ERA5 'w' is in Pa s**-1 !!!
print('omega loaded')
# omega = omega.chunk({"time":2})
# omega
omega = omega[0:2,:,:,:]

omega loaded


In [6]:
# Convert omega to w
w = omega * -9.81 * 0.5  # rho = 0.5 for now, rho to be loaded in properly from file or calculated from temp and pressure
w.attrs["units"] = "m s**-1"
# w

Select length scale $\ell_{max}$

In [7]:
Nlmax = 1

In [8]:
# Dimensions
nt = len(time)
nz = len(lev)
ny = len(lat)
nx = len(lon)

In [9]:
# Horizontal (dR) and vertical (dZ) grid step in m.
dR = abs((lon[0] - lon[1]) * 110000)
dZ = 400  # Suggests interp to 400m grid spacing required

# Horizontal size of the domain
lbox = abs((lon[-1] - lon[0]) * 110000)
# dR

Check for any changes to scripts:

In [10]:
%reload_ext autoreload
%autoreload 2

Run $\verb|CalcPartitionIncrement.py|$

In [11]:
from CalcPartitionIncrement import CalcPartitionIncrement
dR, Nlmax, nphiinc, llx, lly, philsmooth, Nls = CalcPartitionIncrement(dR,Nlmax)

Run $\verb|CalcDRDir_2D.py|$

In [12]:
from CalcDRDir_2D import CalcDRDir_2D
CalcDRDir_2D(dR, Nlmax, u, v, w, nphiinc, llx, lly, philsmooth, Nls)

Average done
SuloDR_np.shape: (1, 2, 12, 721, 1440)
Applying Phil
philsmooth.shape: (1, 1)
SulocDR_np: [[[[[ 1.56765421e-12  1.56765421e-12  1.56765421e-12 ...
      1.56765421e-12  1.56765421e-12  1.56765421e-12]
    [-6.22254089e+02 -6.18762512e+02 -6.15432983e+02 ...
     -6.32203674e+02 -6.28802246e+02 -6.25544067e+02]
    [ 4.01619673e-02  4.06592488e-02  4.06609029e-02 ...
      3.96280289e-02  3.96823026e-02  3.98835018e-02]
    ...
    [ 5.79077809e-04  5.94566925e-04  5.59795648e-04 ...
      6.05319394e-04  6.16171979e-04  5.90855430e-04]
    [ 4.27137391e-04  4.06937615e-04  4.06936713e-04 ...
      4.61921387e-04  4.48220875e-04  4.28883824e-04]
    [ 2.69187393e+01  2.66222897e+01  2.63207378e+01 ...
      2.78225651e+01  2.75117188e+01  2.72217731e+01]]

   [[-2.44946008e-11 -2.44946008e-11 -2.44946008e-11 ...
     -2.44946008e-11 -2.44946008e-11 -2.44946008e-11]
    [-1.87535187e+02 -1.86708557e+02 -1.85809464e+02 ...
     -1.90112320e+02 -1.89228729e+02 -1.88395813e+02]

<xarray.DataArray 'LoSSET_DR' (n_scales: 1, longitude: 1440, latitude: 721,
                               level: 12, time: 2)>
array([[[[[ 1.23742938e-16,  4.17632211e-19],
          [-1.93348370e-15, -2.71640475e-15],
          [ 3.63902774e-14, -8.48753158e-13],
          ...,
          [ 8.74877179e-12,  1.24704396e-12],
          [ 5.21240166e-13,  7.53971175e-12],
          [-6.63185475e-16,  5.02114161e-13]],

         [[-4.91176872e-02, -9.62507136e-02],
          [-1.48031082e-02, -5.32628782e-02],
          [-8.06996045e-03, -7.16191186e-02],
          ...,
          [-1.03220301e-01, -7.08888822e-02],
          [-1.24147208e-01, -8.64486453e-02],
          [-8.66351526e-02, -5.59208770e-02]],

         [[ 3.17018880e-06,  4.57355562e-06],
          [ 9.51687155e-07,  6.27613927e-06],
          [ 7.03614033e-06,  1.79548534e-05],
          ...,
...
          ...,
          [ 1.63396844e-06,  2.31175685e-06],
          [ 1.03193684e-06,  1.49582101e-06],
          [ 1.53303440e-06,  2.10294772e-06]],

         [[ 3.38539865e-08,  7.57701510e-10],
          [ 1.42559581e-07, -1.76145425e-07],
          [ 1.56050877e-07,  5.26939804e-06],
          ...,
          [ 1.19046846e-05,  1.37381999e-05],
          [ 1.08820031e-05,  1.35408365e-05],
          [ 1.06026427e-05,  1.31168805e-05]],

         [[ 2.14875332e-03, -3.07829743e-04],
          [ 1.38292924e-03, -1.88627117e-03],
          [ 3.17110569e-04, -9.37081548e-04],
          ...,
          [ 1.37822660e-02,  2.81103111e-02],
          [ 1.36846318e-02,  2.78658575e-02],
          [ 1.39129336e-02,  2.81408874e-02]]]]])
Coordinates:
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * level      (level) int32 100 150 200 250 300 400 500 600 700 850 925 1000
  * n_scales   (n_scales) int64 0
  * time       (time) datetime64[ns] 2005-01-01T05:00:00 2005-01-01T17:00:00